In [172]:
import pandas as pd
import numpy as np
import dtale
import sweetviz
import io
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import timeit
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold , StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [43]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [180]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [44]:
train['source'] = 'train'
test['source'] = 'test'

In [45]:
df = pd.concat([train, test])

In [46]:
df.shape 

(455495, 19)

In [47]:
df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,source
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10,train
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50,train
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40,train
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50,train
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50,train


# DTALE Used to Analyze EDA as Whole Dataset

In [5]:
dtale.show(df)

2021-05-23 09:58:54,284 - INFO     - NumExpr defaulting to 8 threads.


#EDA For Train Test Comparision

In [26]:
feature_config = sweetviz.FeatureConfig(force_num=["Stay"])
swveda = sweetviz.compare([train, 'Train'], [test, 'Test'], "Stay", feature_config)

                                             |                                             | [  0%]   00:00 ->…

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating th

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating th

In [27]:
swveda.show_html('SWVReport.html')

Report SWVReport.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


C:\ProgramData\Anaconda3\lib\site-packages\dtale\views.py:1723: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\ProgramData\Anaconda3\lib\site-packages\dtale\views.py:1723: FutureWarning:

The default value of regex will change from True to False in a future version.



In [49]:
df['Stay'] = df['Stay'].replace({'0-10' : 0,
                                     '11-20' : 1,
                                     '21-30': 2,
                                     '31-40': 3,
                                     '41-50': 4,
                                     '51-60': 5,
                                     '61-70': 6, 
                                    '71-80': 7,
                                      '81-90': 8,
                                      '91-100': 9,
                                      'More than 100 Days' : 10
                                     })

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455495 entries, 0 to 137056
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            455495 non-null  int64  
 1   Hospital_code                      455495 non-null  int64  
 2   Hospital_type_code                 455495 non-null  int64  
 3   City_Code_Hospital                 455495 non-null  int64  
 4   Hospital_region_code               455495 non-null  int64  
 5   Available Extra Rooms in Hospital  455495 non-null  int64  
 6   Ward_Type                          455495 non-null  int64  
 7   Ward_Facility_Code                 455495 non-null  int64  
 8   Bed Grade                          455495 non-null  int32  
 9   patientid                          455495 non-null  int64  
 10  City_Code_Patient                  448806 non-null  float64
 11  Type of Admission                  4554

In [53]:
df['Hospital_type_code'].value_counts()

a    204730
b     98884
c     66147
e     35428
d     29048
f     15252
g      6006
Name: Hospital_type_code, dtype: int64

In [54]:
df['Hospital_type_code'] = df['Hospital_type_code'].replace({'a' : 0,
                                     'b' : 1,
                                     'c': 2,
                                     'd': 3,
                                     'e': 4,
                                     'f': 5,
                                     'g': 6
                                     })

In [58]:
df['Hospital_region_code'].value_counts()

0    190849
1    174707
2     89939
Name: Hospital_region_code, dtype: int64

In [57]:
df['Hospital_region_code'] = df['Hospital_region_code'].replace({'X' : 0,
                                     'Y' : 1,
                                     'Z': 2})

In [60]:
df['Department'].value_counts()

gynecology            356688
anesthesia             42358
radiotherapy           41033
TB & Chest disease     13751
surgery                 1665
Name: Department, dtype: int64

In [61]:
dept = pd.get_dummies(df['Department'])

In [62]:
dept

,TB & Chest disease,anesthesia,gynecology,radiotherapy,surgery
0,0,0,0,1,0
1,0,0,0,1,0
2,0,1,0,0,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
137052,0,1,0,0,0
137053,0,0,0,1,0
137054,0,1,0,0,0
137055,0,1,0,0,0


In [63]:
df = pd.concat([df, dept], axis = 1)

In [64]:
df.shape

(455495, 24)

In [65]:
df['Ward_Type'].value_counts()

R    182939
Q    152046
S    111166
P      7199
T      2133
U        12
Name: Ward_Type, dtype: int64

In [66]:
df['Ward_Type'] = df['Ward_Type'].replace({'R' : 0,
                                    'Q' : 1,
                                    'S' : 2,
                                    'P' : 3,
                                    'T' : 4,
                                     'U': 5})

In [68]:
del df['Department']

In [70]:
df['Ward_Facility_Code'].value_counts()

F    161470
E     79058
D     74312
C     50279
B     50116
A     40260
Name: Ward_Facility_Code, dtype: int64

In [73]:
df['Bed Grade'].value_counts()

2.0    176451
3.0    158942
4.0     82387
1.0     37567
2.0       148
Name: Bed Grade, dtype: int64

In [71]:
df['Ward_Facility_Code'] = df['Ward_Facility_Code'].replace({'A' : 0,
                                    'B' : 1,
                                    'C' : 2,
                                    'D' : 3,
                                    'E' : 4,
                                     'F': 5})

In [74]:
df['Bed Grade'] = df['Bed Grade'].transform(lambda x: x.fillna('2.0'))

In [80]:
df['Bed Grade'] = df['Bed Grade'].astype(int)

In [82]:
df['Type of Admission'].value_counts()  

Trauma       217672
Emergency    168363
Urgent        69460
Name: Type of Admission, dtype: int64

In [85]:
adtype = pd.get_dummies(df['Type of Admission'])

In [86]:
df = pd.concat([df, adtype], axis = 1)

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455495 entries, 0 to 137056
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            455495 non-null  int64  
 1   Hospital_code                      455495 non-null  int64  
 2   Hospital_type_code                 455495 non-null  int64  
 3   City_Code_Hospital                 455495 non-null  int64  
 4   Hospital_region_code               455495 non-null  int64  
 5   Available Extra Rooms in Hospital  455495 non-null  int64  
 6   Ward_Type                          455495 non-null  int64  
 7   Ward_Facility_Code                 455495 non-null  int64  
 8   Bed Grade                          455495 non-null  int32  
 9   patientid                          455495 non-null  int64  
 10  City_Code_Patient                  448806 non-null  float64
 11  Visitors with Patient              4554

2021-05-23 11:32:09,818 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2021-05-23 11:32:31,211 - INFO     - Executing shutdown...


In [89]:
del df['Type of Admission'] 

In [91]:
df['Severity of Illness'].value_counts()

Moderate    251565
Minor       122735
Extreme      81195
Name: Severity of Illness, dtype: int64

In [92]:
sev = pd.get_dummies(df['Severity of Illness'])

In [94]:
df = pd.concat([df, sev], axis = 1)

In [95]:
df.shape

(455495, 28)

In [97]:
del df['Severity of Illness'] 

In [99]:
df['Age'].value_counts()

41-50     91495
31-40     90420
51-60     69506
21-30     58560
71-80     50737
61-70     48619
11-20     23871
81-90     11240
0-10       9140
91-100     1907
Name: Age, dtype: int64

In [100]:
df['Age'] = df['Age'].replace({'0-10' : 0,
                                     '11-20' : 1,
                                     '21-30': 2,
                                     '31-40': 3,
                                     '41-50': 4,
                                     '51-60': 5,
                                     '61-70': 6, 
                                    '71-80': 7,
                                      '81-90': 8,
                                      '91-100': 9,
                                     })

In [120]:
df['City_Code_Patient'] = df.groupby(['Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Hospital_code']).City_Code_Patient.transform(lambda x: x.fillna(x.median()))

In [121]:
Train = df[df['source'] == 'train']

In [122]:
Test = df[df['source'] == 'test']

In [123]:
Train.shape

(318438, 27)

In [124]:
Test.shape

(137057, 27)

In [125]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318438 entries, 0 to 318437
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  int64  
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  int64  
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Ward_Type                          318438 non-null  int64  
 7   Ward_Facility_Code                 318438 non-null  int64  
 8   Bed Grade                          318438 non-null  int32  
 9   patientid                          318438 non-null  int64  
 10  City_Code_Patient                  318438 non-null  float64
 11  Visitors with Patient              3184

In [133]:
del Train['source']
del Test['source']
del Test['Stay']

In [128]:
yTrain = Train.pop('Stay')

In [135]:
tr = DecisionTreeClassifier(min_samples_leaf= 3,
                                         max_features = 18,
                                         random_state = 100)
tr.fit(Train, yTrain)
trpred_test = tr.predict(Test)
acc = accuracy_score(yTrain,tr.predict(Train))*100

In [136]:
acc

74.16231731137616

In [137]:
trpred_test

array([1., 3., 2., ..., 2., 2., 5.])

In [138]:
predtest = pd.DataFrame(trpred_test, columns =['Stay'])

In [142]:
predtest.value_counts()

Stay              
11-20                 41413
21-30                 36051
31-40                 21018
0-10                  17152
51-60                 11069
41-50                  2917
More than 100 Days     2743
71-80                  2595
81-90                  1354
91-100                  574
61-70                   171
dtype: int64

In [141]:
predtest['Stay'] = predtest['Stay'].replace({0.0 : '0-10',
                                     1.0 : '11-20',
                                     2.0 :'21-30',
                                     3.0 :'31-40',
                                     4.0 :'41-50',
                                     5.0 :'51-60',
                                     6.0 :'61-70', 
                                     7.0 :'71-80',
                                     8.0 : '81-90',
                                     9.0 : '91-100',
                                     10.0 : 'More than 100 Days' 
                                     })

In [161]:
sample = pd.read_csv('sample.csv')

In [145]:
sample.head()

,case_id,Stay
0,318439,0-10
1,318440,0-10
2,318441,0-10
3,318442,0-10
4,318443,0-10


In [146]:
sample['Stay'] = predtest

In [148]:
sample.to_csv('out.csv')

In [155]:
catboost = CatBoostClassifier(eval_metric='Accuracy', max_depth=4, n_estimators=500, verbose=500)
catboost.fit( Train, yTrain )   
 
 

Learning rate set to 0.190699
0:	learn: 0.3383484	total: 304ms	remaining: 2m 31s
499:	learn: 0.4336951	total: 2m 23s	remaining: 0us


In [156]:
pred = catboost.predict(Test)

In [157]:
tpred = catboost.predict(Train)

In [160]:
print(accuracy_score(yTrain,tpred)*100)

43.369509920298455


In [164]:
pred1 = pd.DataFrame(pred, columns =['Stay'])

In [165]:
pred1['Stay'] = pred1['Stay'].replace({0.0 : '0-10',
                                     1.0 : '11-20',
                                     2.0 :'21-30',
                                     3.0 :'31-40',
                                     4.0 :'41-50',
                                     5.0 :'51-60',
                                     6.0 :'61-70', 
                                     7.0 :'71-80',
                                     8.0 : '81-90',
                                     9.0 : '91-100',
                                     10.0 : 'More than 100 Days' 
                                     })

In [168]:
sample['Stay'] = pred1

In [169]:
sample.to_csv('out1.csv')